In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.font_manager import FontProperties
import os 
import sys
from CFST_Cir_Base import CFST_Cir

def FcFy(fc, fy, level, factor, ac, bc, as1, as2, bs):
    fc1 = ac * factor + bc
    fy1 = (as1 * level + as2) * factor + bs
    return fc1*fc, fy1*fy


pathTest = 'D:/Google Drive/Python_Scripts/Test'

#Load SPecimen group
f = open('{0}/Specimen_Group.txt'.format(pathTest), 'r')
groups = [int(item)-1 for item in f.read().split('\n')[0].split(' ')]

f.close()
#Load the specimen data
Conditions = np.loadtxt('{0}/Specimen_Conditions.txt'.format(pathTest))[groups]
#Load the factors
ac, bc = np.loadtxt('factorC.out')
as1, as2, bs = np.loadtxt('factorS1.out')
#Calculate the new fc and fy
output = np.zeros((1,6))
for i in range(len(groups)):
    curNo = groups[i]
    D, temp, t, L, fc, fy, fu, Naxial = Conditions[i]
    curCFST = CFST_Cir(D, t, fc, fy)
    factor = np.log10(curCFST.Ac * curCFST.fc / curCFST.As / curCFST.fy**0.5)
    level = Naxial * 1.0e3 / (curCFST.Ac * curCFST.fc + curCFST.As * curCFST.fy)
    fc1, fy1 = FcFy(fc, fy, level, factor, ac, bc, as1, as2, bs)
    output = np.vstack((output, [D, t, L, fc1, fy1, Naxial]))
output = output[1:]
np.savetxt('./Test/condition_Eq.out', output)